## Importing Librarys and Setting up the Environment 



In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os 
import pandas as pd 
import numpy as np
import copy
import re
import string
path = 'gdrive/MyDrive/2020_21_2/Temalabor_1/processed_popdata/'

## Function definitions
1. Function defintion of trim_all_columns(df) 


In [ ]:
# Function1.: Trimming  all leading and trailing whitespaces, apply str to the whole dfs

def trim_all_columns(df):

    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    
    df = df.applymap(trim_strings)
    df = df.applymap(str) #converting the whole df datatype into string
    return df


## Premliminary steps 

1.   Loading CSVS
2.   Trimming all leading and trailing whitespaces using 

```
trim_all_columns(df) 
```






In [ ]:
# 1. Loading CSVS

full_pop = pd.read_csv(path + 'populationData_HU_1985-2020_full.csv', dtype=str)
basic_pop = pd.read_csv(path + 'populationData_HU_1985-2020_basic.csv', dtype=str)
election=pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/election.csv', dtype=str)


In [ ]:
# 2. Trimming all leading and trailing whitespaces using trim_all_columns(df) aka Using Function1.:

full_pop = trim_all_columns(full_pop)
basic_pop = trim_all_columns(basic_pop)
election = trim_all_columns(election)

## EDA 


### Setting up for election

In [ ]:
elecYear_pop = full_pop[ (full_pop['year'] == '1990') | (full_pop['year'] == '1994') 
                       | (full_pop['year'] == '1998') | (full_pop['year'] == '2002') 
                       | (full_pop['year'] == '2006') | (full_pop['year'] == '2010') 
                       | (full_pop['year'] == '2014') | (full_pop['year'] == '2018') ]

elecYear_pop.reset_index(drop = True, inplace = True)

In [ ]:
elec_years = sorted(elecYear_pop.year.unique())
print('year' + ' ----- ' + 'num of entries' + ' ----- ' + 'unique num of geonameid' + 'unique num of geoid' )
for year in elec_years: 
  print( year, '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]),  
        '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]['geonameid'].unique()), 
        '-----', len(elecYear_pop[(elecYear_pop['year'] == year)]['geoid'].unique()) ) 

year ----- num of entries ----- unique num of geonameidunique num of geoid
1990 ----- 3091 ----- 3091 ----- 3091
1994 ----- 3149 ----- 3149 ----- 3149
1998 ----- 3153 ----- 3153 ----- 3153
2002 ----- 3167 ----- 3167 ----- 3167
2006 ----- 3174 ----- 3174 ----- 3174
2010 ----- 3176 ----- 3176 ----- 3176
2014 ----- 3177 ----- 3177 ----- 3177
2018 ----- 3177 ----- 3177 ----- 3177


In [ ]:
elecYear_pop

,year,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,1990,Aba,4103,1650,9692,1512,47.0343556977497,18.524754962635
1,1994,Aba,4202,1650,9692,1512,47.0343556977497,18.524754962635
2,1998,Aba,4332,1650,9692,1512,47.0343556977497,18.524754962635
3,2002,Aba,4526,1650,9692,1512,47.0343556977497,18.524754962635
4,2006,Aba,4654,1650,9692,1512,47.0343556977497,18.524754962635
...,...,...,...,...,...,...,...,...
25259,2002,Ősi,2244,1650,9759,2160,47.1539446896795,18.190410278261
25260,2006,Ősi,2190,1650,9759,2160,47.1539446896795,18.190410278261
25261,2010,Ősi,2169,1650,9759,2160,47.1539446896795,18.190410278261
25262,2014,Ősi,2089,1650,9759,2160,47.1539446896795,18.190410278261


### **Examining election dfs**



1.   Creating a variable named *year* from stripping *eventdate* for the purpose of merging with **population** on *year* (and *settlement*)
2.   List item



In [ ]:
#1. Creating a variable named year from stripping eventdate 

election=pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/election.csv', dtype=str)
election = trim_all_columns(election)

year = election['eventdate'].str[0:4] 
election['year'] = year 
cols = election.columns.tolist() # listing all the columns, year is the last column at this moment 
cols = cols[-1:] + cols[:-1]  #putting year as a first col 
election = election[cols] #save this into the election df 

In [ ]:
election

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1047408,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0
1047409,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0
1047410,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0
1047411,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0


In [ ]:
# Searching for '16' string in partyids => Pay attention to '166' which is the partyid of Magyar Fuggetlensegi Part 
lst = list()
for i in election.partyids.unique(): 
  if '16' in i:
    lst.append(i)

print(lst)

['16', '166', '269 | 16', '16 | 269', '16 | 26', '26 | 16']


In [ ]:
#Trial: If we had used df.str.match(...), it woud only show us 5 instances instead of 8 in the following case
settlementid996 = election[ (election.settlementid == '996') & ( (election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) ) ]
settlementid996

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
20835,1990,1,1990-03-25,"1990, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,3,7299,54,Fiatal Demokraták Szövetsége,16,26
148260,1994,4,1994-05-08,"1994, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,71178,41,FIDESZ,16,30
282930,1998,7,1998-05-10,"1998, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,8576,25,Fidesz Magyar Polgári Párt,16,30
398760,2002,12,2002-04-16,"2002, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9225,85,MDF | FIDESZ,269 | 16,110
485433,2006,21,2006-04-09,"2006, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,9847,86,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,170
560795,2010,24,2010-04-11,"2010, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 12. választókerület,26361,Hejőszalonta,996,1,67756,83,KDNP | FIDESZ,26 | 16,172
655851,2014,27,2014-04-06,"2014, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,77325,83,FIDESZ | KDNP,16 | 26,188
855990,2018,48,2018-04-08,"2018, listás",Borsod-Abaúj-Zemplén megye,3511,Borsod-Abaúj-Zemplén megye 7. választókerület,31703,Hejőszalonta,996,1,69185,86,Kereszténydemokrata Néppárt | Fidesz Magyar Po...,26 | 16,292


In [ ]:
election.eventdate.unique()  #There are two events in 1998. Need to drop instances in 1998-05-10 which are also in 1998-05-24 

array(['1990-03-25', '1994-05-08', '1998-05-10', '1998-05-24',
       '2002-04-16', '2006-04-09', '2010-04-11', '2014-04-06',
       '2018-04-08'], dtype=object)

In [ ]:
elec98_1stR = election[election['eventdate'] == '1998-05-10'] # Filter only those when eventdate == '1998-05-10'
elec98_2ndR = election[election['eventdate'] == '1998-05-24'] # Filter only those when eventdate == '1998-05-24'. In 98, there were some wards in settlements where 2nd round election had to be held. We are only interested in second round of those. 

wardnameid_lst_elec98_2ndR = list(elec98_2ndR.wardnameid.unique()) # listing the unique wardnameid we come across in the second round of 98's election
wardnameid_dict_elec98_2ndR = {'wardnameid' : wardnameid_lst_elec98_2ndR} #putting them into a dict with 'wardenameid' key. It's essenital when we wanna find those wardnameid from the 2nd round of 98's election in the first round. (We wanna get rid of those in round 1 of 98's election)

In [ ]:
len(wardnameid_lst_elec98_2ndR) #there are 1077 unique wardnameid in the 2nd round of 98s election 

1077

In [ ]:
row_mask = elec98_1stR.isin(wardnameid_dict_elec98_2ndR).any(1)  #PD Series of booelean value where True pertains to values rows where the value of wardnameid_dict_elec98_2ndR can also be found in wardnameid_dict_elec98_2ndR
print( len(elec98_1stR[row_mask].wardnameid.unique()) )  #checking whether we have 1077 unique wardnameid in elec98_1stR[row_mask]. Yes, indeed. 

print( sorted(elec98_1stR[row_mask].wardnameid.unique()) == sorted(elec98_2ndR.wardnameid.unique()) ) #Checking whether the content of elec98_1stR[row_mask].wardnameid.unique() and elec98_2ndR.wardnameid.unique() is the same => yes, indeed.

elec98_1stR_indices_to_delete = elec98_1stR[row_mask].index # we wanna get the indices of elec98_1stR[row_mask] in order to axe them out in the original election df. 
print( len(elec98_1stR_indices_to_delete) ) #There are 12563 rows which contains in both the wardnameid from elec98_1stR and wardnameid from elec98_2ndR

1077
True
12563


In [ ]:
election = election.drop(elec98_1stR_indices_to_delete) #Dropping those indices from election df and save it again in election  
election.reset_index(drop = True, inplace = True) # reset the indexing of the new election 

In [ ]:
election

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034845,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0
1034846,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0
1034847,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0
1034848,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0


In [ ]:
# NEW

electionPopulation =  pd.merge(election, elecYear_pop, left_on=['settlementid', 'year'], right_on=['geoid', 'year'])
votePop_elig = electionPopulation['value'].astype(np.int64) * 0.8
votePop_elig = votePop_elig.round().astype(np.int64)
electionPopulation['votePop_elig'] = votePop_elig
electionPopulation = electionPopulation.applymap(str)
electionPopulation

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag,votePop_elig
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034845,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034846,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034847,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
1034848,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56


In [ ]:
len(electionPopulation.name.unique())

3188

### Splitting up the merged electionPopulation df

Fidesz's *partyid==16* so let's dissect our **election** df into **FIDESZ**  **NotFidesz** dfs



```
(election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) 
```



In [ ]:
eFidesz = election[ (election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) ]
eNotFidesz = election[~( (election.partyids.str.contains('^16$')) | (election.partyids.str.contains('^16 \|')) | (election.partyids.str.contains('\| 16$')) ) ]

In [ ]:
print(len(eFidesz)) #number of rows in eFidesz df 
print(len(eNotFidesz)) ##number of rows in eNotFidesz df
print(len(eFidesz) + len(eNotFidesz) == len(election))  #Check whether the sum of splitted dfs equal to the original 

86033
948817
True


As we can see there are significant data losses in **electionpopulation** compared to **election**. 
**electionpoulation** includes poulation data and other data as well. 

In [ ]:
Fidesz = electionPopulation[ (electionPopulation.partyids.str.contains('^16$')) | (electionPopulation.partyids.str.contains('^16 \|')) | (electionPopulation.partyids.str.contains('\| 16$')) ]
Fidesz.reset_index(drop=True, inplace=True)

NotFidesz = electionPopulation[~( (electionPopulation.partyids.str.contains('^16$')) | (electionPopulation.partyids.str.contains('^16 \|')) | (electionPopulation.partyids.str.contains('\| 16$')) ) ]
NotFidesz.reset_index(drop=True, inplace=True)

print(len(Fidesz)) #number of rows in Fidesz df 
print(len(NotFidesz)) ##number of rows in NotFidesz df
print(len(Fidesz) + len(NotFidesz) == len(electionPopulation))  #Check whether the sum of splitted dfs equal to the original 

86033
948817
True


In [ ]:
Fidesz

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag,votePop_elig
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,54,Fiatal Demokraták Szövetsége,16,23,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,2,13661,54,Fiatal Demokraták Szövetsége,16,16,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,3,13662,54,Fiatal Demokraták Szövetsége,16,27,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,4,13663,54,Fiatal Demokraták Szövetsége,16,34,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,5,13664,54,Fiatal Demokraták Szövetsége,16,30,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86028,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalasárszeg,97,1,86023,86,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,37,Zalasárszeg,127,1650,15763,97,46.4946504867345,17.0821631084216,102
86029,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentbalázs,1628,1,11834,86,Kereszténydemokrata Néppárt | Fidesz Magyar Po...,26 | 16,223,Zalaszentbalázs,798,1650,15762,1628,46.5909416707031,16.9189750148023,638
86030,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaszentjakab,1581,1,86026,86,Kereszténydemokrata Néppárt | Fidesz Magyar Po...,26 | 16,98,Zalaszentjakab,320,1650,17105,1581,46.486657871767,17.1297592011168,256
86031,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zalaújlak,723,1,86031,86,Fidesz Magyar Polgári Szövetség | Keresztényde...,16 | 26,37,Zalaújlak,104,1650,17104,723,46.5607974894449,17.0798804702104,83


In [ ]:
NotFidesz

,year,event_id,eventdate,eventtype,county,countyid,electdistrict,edid,settlement,settlementid,ward,wardnameid,partylistid,partylist,partyids,vote,name,value,datatypeid,geonameid,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag,votePop_elig
0,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,3,Hazafias Választási Koalíció,133,0,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
1,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,6,Kereszténydemokrata Néppárt,26,13,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
2,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,29,Szabad Demokraták Szövetsége,240,45,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
3,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,31,Magyarországi Szociáldemokrata Párt,52,15,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
4,1990,1,1990-03-25,"1990, listás",Bács-Kiskun megye,3508,Bács-Kiskun megye 10. választókerület,26342,Bácsalmás,855,1,13659,35,Magyar Szocialista Munkáspárt,182,14,Bácsalmás,8267,1650,10491,855,46.1291312906161,19.3285702772085,6614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948812,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,223,Közös Nevező,60744,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
948813,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,224,Sportos és Egészséges Magyarországért Párt,60753,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
948814,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,225,Net Párt,60724,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56
948815,2018,48,2018-04-08,"2018, listás",Zala megye,3526,Zala megye 3. választókerület,31798,Zebecke,169,1,79252,226,Európai Roma Keresztények Jobblétéért Demokrat...,60718,0,Zebecke,70,1650,10403,169,46.6433081664363,16.6861596166253,56


In [ ]:
#Checking the unique partyids => it show us what we want to achieve 
Fidesz.partyids.unique()

array(['16', '269 | 16', '16 | 269', '16 | 26', '26 | 16'], dtype=object)

In [ ]:
#Checking whether any partidys pertaining to Fidesz leaked to NotFidesz dataframe => For god sake, no. Only 166, but this is the party id of Magyar Fuggetlensegi Part. So far, so good. 
lst2 = list()
for i in NotFidesz.partyids.unique(): 
  if '16' in i:
    lst2.append(i)

print(lst2)

['166']


In [ ]:
#Checking the # of unique geoids 
len(NotFidesz.geoid.unique()) == len(Fidesz.geoid.unique())

True

In [ ]:
#Comparing the # of unique settlementid in both Fidesz and NotFidesz by election years 
print('1990 -----', len(Fidesz[Fidesz.year == '1990'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '1990'].settlementid.unique()))
print('1994 -----', len(Fidesz[Fidesz.year == '1994'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '1994'].settlementid.unique()))
print('1998 -----', len(Fidesz[Fidesz.year == '1998'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '1998'].settlementid.unique()))
print('2002 -----', len(Fidesz[Fidesz.year == '2002'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2002'].settlementid.unique()))
print('2006 -----', len(Fidesz[Fidesz.year == '2006'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2006'].settlementid.unique()))
print('2010 -----', len(Fidesz[Fidesz.year == '2010'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2010'].settlementid.unique()))
print('2014 -----', len(Fidesz[Fidesz.year == '2014'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2014'].settlementid.unique()))
print('2018 -----', len(Fidesz[Fidesz.year == '2018'].settlementid.unique()), len(NotFidesz[NotFidesz.year == '2018'].settlementid.unique()))

1990 ----- 3089 3089
1994 ----- 3135 3135
1998 ----- 3153 3153
2002 ----- 3157 3157
2006 ----- 3167 3167
2010 ----- 3174 3174
2014 ----- 3176 3176
2018 ----- 3177 3177


In [ ]:
#Comparing the CONTENTS of uniduqe settlementids in both Fidesz and NotFidesz => turns out all of them have the same settlementid contents in every election years

print('1990 -----', sorted(Fidesz[Fidesz.year == '1990'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '1990'].settlementid.unique()) )
print('1994 -----', sorted(Fidesz[Fidesz.year == '1994'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '1994'].settlementid.unique()) )
print('1998 -----', sorted(Fidesz[Fidesz.year == '1998'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '1998'].settlementid.unique()) )
print('2002 -----', sorted(Fidesz[Fidesz.year == '2002'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2002'].settlementid.unique()) )
print('2006 -----', sorted(Fidesz[Fidesz.year == '2006'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2006'].settlementid.unique()) )
print('2010 -----', sorted(Fidesz[Fidesz.year == '2010'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2010'].settlementid.unique()) )
print('2014 -----', sorted(Fidesz[Fidesz.year == '2014'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2014'].settlementid.unique()) )
print('2018 -----', sorted(Fidesz[Fidesz.year == '2018'].settlementid.unique()) == sorted(NotFidesz[NotFidesz.year == '2018'].settlementid.unique()) )

1990 ----- True
1994 ----- True
1998 ----- True
2002 ----- True
2006 ----- True
2010 ----- True
2014 ----- True
2018 ----- True


### ```settlementid```-ra vonatkozo **nepessegszamot** es **szavazasra jogosultak szamat** es az aktualis szavazat szamot is vedd bele, ami a ```value```, ```votePop_elig``` es  ```vote``` oszlop alatt van letarolva azt belekene venni az alabbi adattablakba a szemleletesseg kedveert. 

### Illetve a kulonbozo szavazati aranyok szamolasan kerlek hozz letre valami uj oszlopnevet az aranyossagra, mert a ```vote``` tartsuk meg az aktualis szavazatszamra

### Some basic calculations related to votes 

In [ ]:
trunc_elecYear_pop = copy.deepcopy(elecYear_pop)
trunc_elecYear_pop = trunc_elecYear_pop[['year', 'geonameid', 'geoid', 'name', 'value', 'datatypeid', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag']].drop_duplicates()
trunc_electionPopulation = electionPopulation[['year', 'settlementid', 'settlement', 'value', 'county', 'votePop_elig', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag']].drop_duplicates()

In [ ]:
# Alternativ: merging WITH on name/settlement too 

election_pop_small_alt = trunc_elecYear_pop.merge(trunc_electionPopulation, left_on=['year', 'geoid', 'value', 'name', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag'], right_on=['year', 'settlementid', 'value', 'settlement', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag'])
temp_election_pop_small_alt = election_pop_small_alt[['year', 'geoid', 'settlementid', 'name', 'settlement', 'value']]
temp_election_pop_small_alt

,year,geoid,settlementid,name,settlement,value
0,1990,1512,1512,Aba,Aba,4103
1,1994,1512,1512,Aba,Aba,4202
2,1998,1512,1512,Aba,Aba,4332
3,2002,1512,1512,Aba,Aba,4526
4,2006,1512,1512,Aba,Aba,4654
...,...,...,...,...,...,...
25194,2002,2160,2160,Ősi,Ősi,2244
25195,2006,2160,2160,Ősi,Ősi,2190
25196,2010,2160,2160,Ősi,Ősi,2169
25197,2014,2160,2160,Ősi,Ősi,2089


In [ ]:
# merging WITHOUT on name/settlement too 

election_pop_small = trunc_elecYear_pop.merge(trunc_electionPopulation, left_on=['year', 'geoid', 'value', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag'], right_on=['year', 'settlementid', 'value', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag'])
temp_election_pop_small = election_pop_small[['year', 'geoid', 'settlementid', 'name', 'settlement', 'value']]
temp_election_pop_small

,year,geoid,settlementid,name,settlement,value
0,1990,1512,1512,Aba,Aba,4103
1,1994,1512,1512,Aba,Aba,4202
2,1998,1512,1512,Aba,Aba,4332
3,2002,1512,1512,Aba,Aba,4526
4,2006,1512,1512,Aba,Aba,4654
...,...,...,...,...,...,...
25223,2002,2160,2160,Ősi,Ősi,2244
25224,2006,2160,2160,Ősi,Ősi,2190
25225,2010,2160,2160,Ősi,Ősi,2169
25226,2014,2160,2160,Ősi,Ősi,2089


In [ ]:
name_settlement_diff = temp_election_pop_small.merge(temp_election_pop_small_alt, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
print(len(name_settlement_diff)) #29 = 25228 - 25199 (aka: temp_election_pop_small - temp_election_pop_small_alt)
name_settlement_diff #from now on we opt for name instead of settlement

29


,year,geoid,settlementid,name,settlement,value,_merge
1420,1990,1811,1811,Baranyahídvég,Baranyahidvég,213,left_only
1421,1994,1811,1811,Baranyahídvég,Baranyahidvég,221,left_only
1422,1998,1811,1811,Baranyahídvég,Baranyahidvég,231,left_only
1423,2002,1811,1811,Baranyahídvég,Baranyahidvég,232,left_only
6437,1990,1428,1428,Fűzvölgy,Füzvölgy,165,left_only
6438,1994,1428,1428,Fűzvölgy,Füzvölgy,162,left_only
6439,1998,1428,1428,Fűzvölgy,Füzvölgy,147,left_only
6440,2002,1428,1428,Fűzvölgy,Füzvölgy,141,left_only
7655,1990,554,554,Harka,Magyarfalva,1079,left_only
12994,1990,2827,2827,Megyehíd,Megyehid,330,left_only


In [ ]:
election_pop_small.head(1)

,year,geonameid,geoid,name,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,settlementid,settlement,county,votePop_elig
0,1990,9692,1512,Aba,4103,1650,47.0343556977497,18.524754962635,1512,Aba,Fejér megye,3282


In [ ]:
election_pop_small = election_pop_small.drop(columns=['settlement']) # Just retain name column 

In [ ]:
election_pop_small.head(1)

,year,geonameid,geoid,name,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,settlementid,county,votePop_elig
0,1990,9692,1512,Aba,4103,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3282


In [ ]:
missingFrom_election_pop_small = trunc_elecYear_pop.merge(election_pop_small[trunc_elecYear_pop.columns], how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
print(len(missingFrom_election_pop_small))
missingFrom_election_pop_small

36


,year,geonameid,geoid,name,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,_merge
326,1994,16296,3157,Alsóbogát,345,1650,46.5112897587233,17.7504681543824,left_only
1066,1994,10343,2012,Balajt,459,1650,48.3212917188831,20.7880871476492,left_only
1097,2014,10696,16544,Balatonakarattya,893,1650,47.0215257290545,18.1557677431017,left_only
2643,1994,10443,3153,Budapest XXIII. kerület,20450,1650,47.3962135225183,19.1545494172079,left_only
3511,1994,16660,3154,Csataszög,303,1650,47.2832586476358,20.3837622266224,left_only
3814,1994,9934,3143,Csincse,675,1650,47.8893574421003,20.7692021468904,left_only
4037,1990,5452,2300,Csárdaszállás,624,1650,46.864668817184,20.9370441583545,left_only
4244,2002,10445,3241,Csörög,1650,1650,47.7344779491825,19.1928394255503,left_only
4368,2002,17269,3227,Daruszentmiklós,1372,1650,46.873221124716,18.8583746470557,left_only
6766,2006,780,3191,Gibárt,372,1650,48.3140107206399,21.1649090810549,left_only


In [ ]:
election_pop_small = election_pop_small.rename(columns={"name": "settlement"}) # Raname the name column to settlement

In [ ]:
sorted(election_pop_small.settlementid) == sorted(election_pop_small.geoid)

True

In [ ]:
#election_pop_small = election_pop_small.rename(columns={"name": "settlement"}) # Rename the name column to settlement
election_pop_small = election_pop_small.drop(columns=['geoid']) #Remove geoid because it is the same as settlementid
election_pop_small

,year,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,settlementid,county,votePop_elig
0,1990,9692,Aba,4103,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3282
1,1994,9692,Aba,4202,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3362
2,1998,9692,Aba,4332,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3466
3,2002,9692,Aba,4526,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3621
4,2006,9692,Aba,4654,1650,47.0343556977497,18.524754962635,1512,Fejér megye,3723
...,...,...,...,...,...,...,...,...,...,...
25223,2002,9759,Ősi,2244,1650,47.1539446896795,18.190410278261,2160,Veszprém megye,1795
25224,2006,9759,Ősi,2190,1650,47.1539446896795,18.190410278261,2160,Veszprém megye,1752
25225,2010,9759,Ősi,2169,1650,47.1539446896795,18.190410278261,2160,Veszprém megye,1735
25226,2014,9759,Ősi,2089,1650,47.1539446896795,18.190410278261,2160,Veszprém megye,1671


In [ ]:
NotFidesz['vote']=NotFidesz['vote'].astype(np.int64)           #hova lettek az adatok?
votes_NotFidesz=NotFidesz[['year','settlementid','vote']].groupby(['year','settlementid']).sum()
votes_NotFidesz=votes_NotFidesz.reset_index()

votes_NotFidesz_pop=pd.merge(votes_NotFidesz, election_pop_small, on=['settlementid', 'year'])
votes_NotFidesz_pop.rename(columns={"vote": "vote_NF"}, inplace = True)
votes_NotFidesz_pop

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,settlementid,vote_NF,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig
0,1990,10,827,16316,Kakasd,1786,1650,46.3459647218548,18.5956807919195,Tolna megye,1429
1,1990,100,44078,17145,Szombathely,85350,1650,47.2335230401357,16.6203252254345,Vas megye,68280
2,1990,1000,1314,11159,Romhány,2953,1650,47.922803401609,19.2656889662957,Nógrád megye,2362
3,1990,1001,113,10400,Zók,293,1650,46.0095771873554,18.0999673899155,Baranya megye,234
4,1990,1002,1037,392,Tállya,2531,1650,48.2359357513637,21.2264437638785,Borsod-Abaúj-Zemplén megye,2025
...,...,...,...,...,...,...,...,...,...,...,...
25223,2018,995,581,16952,Csepreg,3243,1650,47.4027100951984,16.7109602630874,Vas megye,2594
25224,2018,996,124,16328,Hejőszalonta,851,1650,47.9386636958907,20.8821611468759,Borsod-Abaúj-Zemplén megye,681
25225,2018,997,17,5291,Patca,57,1650,46.2844202850219,17.7252774001829,Somogy megye,46
25226,2018,998,264,16310,Kardoskút,819,1650,46.4981256900021,20.7026923838598,Békés megye,655


In [ ]:
Fidesz['vote']=Fidesz['vote'].astype(np.int64)
votes_Fidesz=Fidesz[['year','settlementid','vote']].groupby(['year','settlementid']).sum()
votes_Fidesz=votes_Fidesz.reset_index()
votes_Fidesz_pop=pd.merge(votes_Fidesz, election_pop_small, on=['settlementid', 'year'])
votes_Fidesz_pop.rename(columns={"vote": "vote_F"}, inplace = True)
votes_Fidesz_pop

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,year,settlementid,vote_F,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig
0,1990,10,102,16316,Kakasd,1786,1650,46.3459647218548,18.5956807919195,Tolna megye,1429
1,1990,100,3563,17145,Szombathely,85350,1650,47.2335230401357,16.6203252254345,Vas megye,68280
2,1990,1000,176,11159,Romhány,2953,1650,47.922803401609,19.2656889662957,Nógrád megye,2362
3,1990,1001,6,10400,Zók,293,1650,46.0095771873554,18.0999673899155,Baranya megye,234
4,1990,1002,79,392,Tállya,2531,1650,48.2359357513637,21.2264437638785,Borsod-Abaúj-Zemplén megye,2025
...,...,...,...,...,...,...,...,...,...,...,...
25223,2018,995,1221,16952,Csepreg,3243,1650,47.4027100951984,16.7109602630874,Vas megye,2594
25224,2018,996,292,16328,Hejőszalonta,851,1650,47.9386636958907,20.8821611468759,Borsod-Abaúj-Zemplén megye,681
25225,2018,997,21,5291,Patca,57,1650,46.2844202850219,17.7252774001829,Somogy megye,46
25226,2018,998,188,16310,Kardoskút,819,1650,46.4981256900021,20.7026923838598,Békés megye,655


In [ ]:
votes_NotFidesz_per_Fidesz = copy.deepcopy(votes_NotFidesz_pop)
votes_NotFidesz_per_Fidesz['vote_rate_NF_per_F']=votes_NotFidesz['vote']/votes_Fidesz['vote']
votes_NotFidesz_per_Fidesz.drop(columns=['vote_NF'], inplace=True)
votes_NotFidesz_per_Fidesz

,year,settlementid,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_NF_per_F
0,1990,10,16316,Kakasd,1786,1650,46.3459647218548,18.5956807919195,Tolna megye,1429,8.107843
1,1990,100,17145,Szombathely,85350,1650,47.2335230401357,16.6203252254345,Vas megye,68280,12.371036
2,1990,1000,11159,Romhány,2953,1650,47.922803401609,19.2656889662957,Nógrád megye,2362,7.465909
3,1990,1001,10400,Zók,293,1650,46.0095771873554,18.0999673899155,Baranya megye,234,18.833333
4,1990,1002,392,Tállya,2531,1650,48.2359357513637,21.2264437638785,Borsod-Abaúj-Zemplén megye,2025,13.126582
...,...,...,...,...,...,...,...,...,...,...,...
25223,2018,995,16952,Csepreg,3243,1650,47.4027100951984,16.7109602630874,Vas megye,2594,0.475839
25224,2018,996,16328,Hejőszalonta,851,1650,47.9386636958907,20.8821611468759,Borsod-Abaúj-Zemplén megye,681,0.424658
25225,2018,997,5291,Patca,57,1650,46.2844202850219,17.7252774001829,Somogy megye,46,0.809524
25226,2018,998,16310,Kardoskút,819,1650,46.4981256900021,20.7026923838598,Békés megye,655,1.404255


In [ ]:
votes_rating_Fidesz = copy.deepcopy(votes_Fidesz_pop)
votes_rating_Fidesz['vote_rate_F'] = votes_Fidesz['vote'] / (votes_NotFidesz['vote']+votes_Fidesz['vote'])
votes_rating_Fidesz

,year,settlementid,vote_F,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_F
0,1990,10,102,16316,Kakasd,1786,1650,46.3459647218548,18.5956807919195,Tolna megye,1429,0.109795
1,1990,100,3563,17145,Szombathely,85350,1650,47.2335230401357,16.6203252254345,Vas megye,68280,0.074789
2,1990,1000,176,11159,Romhány,2953,1650,47.922803401609,19.2656889662957,Nógrád megye,2362,0.118121
3,1990,1001,6,10400,Zók,293,1650,46.0095771873554,18.0999673899155,Baranya megye,234,0.050420
4,1990,1002,79,392,Tállya,2531,1650,48.2359357513637,21.2264437638785,Borsod-Abaúj-Zemplén megye,2025,0.070789
...,...,...,...,...,...,...,...,...,...,...,...,...
25223,2018,995,1221,16952,Csepreg,3243,1650,47.4027100951984,16.7109602630874,Vas megye,2594,0.677580
25224,2018,996,292,16328,Hejőszalonta,851,1650,47.9386636958907,20.8821611468759,Borsod-Abaúj-Zemplén megye,681,0.701923
25225,2018,997,21,5291,Patca,57,1650,46.2844202850219,17.7252774001829,Somogy megye,46,0.552632
25226,2018,998,188,16310,Kardoskút,819,1650,46.4981256900021,20.7026923838598,Békés megye,655,0.415929


In [ ]:
votes_rating_NotFidesz = copy.deepcopy(votes_NotFidesz_pop)
votes_rating_NotFidesz['vote_rate_NF'] = votes_NotFidesz['vote']/(votes_NotFidesz['vote']+votes_Fidesz['vote'])
votes_rating_NotFidesz

,year,settlementid,vote_NF,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_NF
0,1990,10,827,16316,Kakasd,1786,1650,46.3459647218548,18.5956807919195,Tolna megye,1429,0.890205
1,1990,100,44078,17145,Szombathely,85350,1650,47.2335230401357,16.6203252254345,Vas megye,68280,0.925211
2,1990,1000,1314,11159,Romhány,2953,1650,47.922803401609,19.2656889662957,Nógrád megye,2362,0.881879
3,1990,1001,113,10400,Zók,293,1650,46.0095771873554,18.0999673899155,Baranya megye,234,0.949580
4,1990,1002,1037,392,Tállya,2531,1650,48.2359357513637,21.2264437638785,Borsod-Abaúj-Zemplén megye,2025,0.929211
...,...,...,...,...,...,...,...,...,...,...,...,...
25223,2018,995,581,16952,Csepreg,3243,1650,47.4027100951984,16.7109602630874,Vas megye,2594,0.322420
25224,2018,996,124,16328,Hejőszalonta,851,1650,47.9386636958907,20.8821611468759,Borsod-Abaúj-Zemplén megye,681,0.298077
25225,2018,997,17,5291,Patca,57,1650,46.2844202850219,17.7252774001829,Somogy megye,46,0.447368
25226,2018,998,264,16310,Kardoskút,819,1650,46.4981256900021,20.7026923838598,Békés megye,655,0.584071


In [ ]:
#Why 

votes_Fidesz['vote'] = votes_Fidesz['vote'].astype(str)
votes_NotFidesz['vote'] = votes_NotFidesz['vote'].astype(str)

#votes_NotFidesz_per_Fidesz['vote'] = votes_NotFidesz_per_Fidesz['vote'].astype(str)

votes_rating_NotFidesz['vote_NF'] = votes_rating_NotFidesz['vote_NF'].astype(str)
votes_rating_Fidesz['vote_F'] = votes_rating_Fidesz['vote_F'].astype(str)

In [ ]:
sanity_check_vote=pd.merge(votes_rating_NotFidesz, votes_rating_Fidesz, on=['year', 'settlementid', 'geonameid', 'settlement', 'value', 'datatypeid', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag', 'county','votePop_elig'])
checking_rating=sanity_check_vote['vote_rate_NF']+sanity_check_vote['vote_rate_F']
checking_rating=checking_rating.reset_index()
print(len(checking_rating[checking_rating[0]==1]))
print(len(checking_rating))

25228
25228


In [ ]:
sanity_check_vote

,year,settlementid,vote_NF,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_NF,vote_F,vote_rate_F
0,1990,10,827,16316,Kakasd,1786,1650,46.3459647218548,18.5956807919195,Tolna megye,1429,0.890205,102,0.109795
1,1990,100,44078,17145,Szombathely,85350,1650,47.2335230401357,16.6203252254345,Vas megye,68280,0.925211,3563,0.074789
2,1990,1000,1314,11159,Romhány,2953,1650,47.922803401609,19.2656889662957,Nógrád megye,2362,0.881879,176,0.118121
3,1990,1001,113,10400,Zók,293,1650,46.0095771873554,18.0999673899155,Baranya megye,234,0.949580,6,0.050420
4,1990,1002,1037,392,Tállya,2531,1650,48.2359357513637,21.2264437638785,Borsod-Abaúj-Zemplén megye,2025,0.929211,79,0.070789
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25223,2018,995,581,16952,Csepreg,3243,1650,47.4027100951984,16.7109602630874,Vas megye,2594,0.322420,1221,0.677580
25224,2018,996,124,16328,Hejőszalonta,851,1650,47.9386636958907,20.8821611468759,Borsod-Abaúj-Zemplén megye,681,0.298077,292,0.701923
25225,2018,997,17,5291,Patca,57,1650,46.2844202850219,17.7252774001829,Somogy megye,46,0.447368,21,0.552632
25226,2018,998,264,16310,Kardoskút,819,1650,46.4981256900021,20.7026923838598,Békés megye,655,0.584071,188,0.415929


In [ ]:
sanity_check_vote['vote_NF']=sanity_check_vote['vote_NF'].astype(np.int64)
sanity_check_vote['vote_F']=sanity_check_vote['vote_F'].astype(np.int64)
sanity_check_vote['value']=sanity_check_vote['value'].astype(np.int64)
sanity_check_vote['votePop_elig']=sanity_check_vote['votePop_elig'].astype(np.int64)
sanity_check_vote['sumvote']=sanity_check_vote['vote_NF']+sanity_check_vote['vote_F']
sanity_check_vote['vote_rate_NF_per_F'] = sanity_check_vote['vote_NF'] / sanity_check_vote['vote_F']
sanity_check_vote #Ennek az oszlopait majd a konnyen olvashatosag kedveert kellene rendezni, amikor a klaszter vizsgalatkor betoltjuk ezt a tablazatot

,year,settlementid,vote_NF,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_NF,vote_F,vote_rate_F,sumvote,vote_rate_NF_per_F
0,1990,10,827,16316,Kakasd,1786,1650,46.3459647218548,18.5956807919195,Tolna megye,1429,0.890205,102,0.109795,929,8.107843
1,1990,100,44078,17145,Szombathely,85350,1650,47.2335230401357,16.6203252254345,Vas megye,68280,0.925211,3563,0.074789,47641,12.371036
2,1990,1000,1314,11159,Romhány,2953,1650,47.922803401609,19.2656889662957,Nógrád megye,2362,0.881879,176,0.118121,1490,7.465909
3,1990,1001,113,10400,Zók,293,1650,46.0095771873554,18.0999673899155,Baranya megye,234,0.949580,6,0.050420,119,18.833333
4,1990,1002,1037,392,Tállya,2531,1650,48.2359357513637,21.2264437638785,Borsod-Abaúj-Zemplén megye,2025,0.929211,79,0.070789,1116,13.126582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25223,2018,995,581,16952,Csepreg,3243,1650,47.4027100951984,16.7109602630874,Vas megye,2594,0.322420,1221,0.677580,1802,0.475839
25224,2018,996,124,16328,Hejőszalonta,851,1650,47.9386636958907,20.8821611468759,Borsod-Abaúj-Zemplén megye,681,0.298077,292,0.701923,416,0.424658
25225,2018,997,17,5291,Patca,57,1650,46.2844202850219,17.7252774001829,Somogy megye,46,0.447368,21,0.552632,38,0.809524
25226,2018,998,264,16310,Kardoskút,819,1650,46.4981256900021,20.7026923838598,Békés megye,655,0.584071,188,0.415929,452,1.404255


In [ ]:
sanity_check_vote[sanity_check_vote['votePop_elig']<sanity_check_vote['sumvote']] #For the curiosity, just checking settlements where the totalvote is greater than the votePop_elig

,year,settlementid,vote_NF,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_NF,vote_F,vote_rate_F,sumvote,vote_rate_NF_per_F
187,1990,117,28,16528,Iborfia,31,1650,46.7413545031963,16.7506574016514,Zala megye,25,1.000000,0,0.000000,28,inf
202,1990,1183,33,17411,Sénye,43,1650,46.8952860476735,17.1349972253716,Zala megye,34,0.942857,2,0.057143,35,16.500000
356,1990,1321,109,10233,Nagyhuta,131,1650,48.4286008097919,21.495086224793,Borsod-Abaúj-Zemplén megye,105,0.964602,4,0.035398,113,27.250000
415,1990,1375,41,16181,Dötk,41,1650,46.9440774536757,17.0093471009652,Zala megye,33,1.000000,0,0.000000,41,inf
533,1990,1481,41,16894,Keresztéte,51,1650,48.5004218033779,20.9530798606733,Borsod-Abaúj-Zemplén megye,41,0.976190,1,0.023810,42,41.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22239,2018,117,2,16528,Iborfia,8,1650,46.7413545031963,16.7506574016514,Zala megye,6,0.250000,6,0.750000,8,0.333333
22721,2018,1604,3511,15941,Vásárosnamény,9055,1650,48.1285832062129,22.3136287446675,Szabolcs-Szatmár-Bereg megye,7244,0.460640,4111,0.539360,7622,0.854050
23719,2018,2503,16,17328,Zalaszombatfa,50,1650,46.6388295103915,16.4457057391619,Zala megye,40,0.390244,25,0.609756,41,0.640000
24854,2018,662,10,10270,Alsószenterzsébet,57,1650,46.744556760313,16.4774152582769,Zala megye,46,0.208333,38,0.791667,48,0.263158


In [ ]:
sanity_check_vote[sanity_check_vote['value']<sanity_check_vote['sumvote']] #Throwing those indexes from sanity_check_vote where total vote exceed the population number. 

,year,settlementid,vote_NF,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_NF,vote_F,vote_rate_F,sumvote,vote_rate_NF_per_F
3599,1994,146,260,16176,Edde,268,1650,46.524660354456,17.7179759182014,Somogy megye,214,0.915493,24,0.084507,284,10.833333
4305,1994,2098,23,9795,Sima,23,1650,48.3002395294024,21.3050886391276,Borsod-Abaúj-Zemplén megye,18,0.920000,2,0.080000,25,11.500000
17844,2010,2932,6,16581,Debréte,19,1650,48.4978247736252,20.8654470658118,Borsod-Abaúj-Zemplén megye,15,0.300000,14,0.700000,20,0.428571


In [ ]:
idxToDelFrom_sanity_check_vote = sanity_check_vote[sanity_check_vote['value']<sanity_check_vote['sumvote']].index
sanity_check_vote = sanity_check_vote.drop(idxToDelFrom_sanity_check_vote)
sanity_check_vote.sort_values(by=['year', 'settlement'], inplace = True)
sanity_check_vote.reset_index(drop = True, inplace = True)

In [ ]:
sanity_check_vote

,year,settlementid,vote_NF,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_NF,vote_F,vote_rate_F,sumvote,vote_rate_NF_per_F
0,1990,1512,1456,9692,Aba,4103,1650,47.0343556977497,18.524754962635,Fejér megye,3282,0.951634,74,0.048366,1530,19.675676
1,1990,1035,255,16239,Abaliget,645,1650,46.1438461930372,18.1181108415909,Baranya megye,516,0.958647,11,0.041353,266,23.181818
2,1990,2209,1568,10360,Abasár,3347,1650,47.8006871447644,20.0083983387857,Heves megye,2678,0.912158,151,0.087842,1719,10.384106
3,1990,1341,89,17191,Abaújalpár,136,1650,48.3061466428872,21.234096519055,Borsod-Abaúj-Zemplén megye,109,0.946809,5,0.053191,94,17.800000
4,1990,2420,353,10500,Abaújkér,709,1650,48.3050378371762,21.2002207816169,Borsod-Abaúj-Zemplén megye,567,0.928947,27,0.071053,380,13.074074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25220,2018,1122,44,16102,Őrimagyarósd,229,1650,46.8890265030192,16.5370706560826,Vas megye,183,0.312057,97,0.687943,141,0.453608
25221,2018,847,339,115,Őriszentpéter,1166,1650,46.8450146449415,16.4221839964927,Vas megye,933,0.476124,373,0.523876,712,0.908847
25222,2018,1177,84,10072,Őrtilos,493,1650,46.2822175115168,16.9269824353792,Somogy megye,394,0.333333,168,0.666667,252,0.500000
25223,2018,1701,73,16101,Ősagárd,336,1650,47.8589621562394,19.1935550499782,Nógrád megye,269,0.370558,124,0.629442,197,0.588710


In [ ]:
Budapest_distr = sanity_check_vote[sanity_check_vote['settlement'].str.contains('Budapest')] #Should Give us 182 because district XXIII. was formed after the 1994th election 
Budapest_distr

,year,settlementid,vote_NF,geonameid,settlement,value,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,county,votePop_elig,vote_rate_NF,vote_F,vote_rate_F,sumvote,vote_rate_NF_per_F
299,1990,741,20305,9742,Budapest I. kerület,35305,1650,47.4965189917372,19.0347155108027,Budapest,28244,0.885367,2629,0.114633,22934,7.723469
300,1990,116,55615,9743,Budapest II. kerület,100441,1650,47.5444115110089,18.9751760210319,Budapest,80353,0.885787,7171,0.114213,62786,7.755543
301,1990,1579,66401,9744,Budapest III. kerület,142862,1650,47.5699972451916,19.0390024567837,Budapest,114290,0.884651,8658,0.115349,75059,7.669323
302,1990,341,42701,1020,Budapest IV. kerület,109509,1650,47.580097299462,19.0972915873203,Budapest,87607,0.876439,6020,0.123561,48721,7.093189
303,1990,2701,34743,15726,Budapest IX. kerület,74746,1650,47.468240300406,19.0934214369353,Budapest,59797,0.879147,4776,0.120853,39519,7.274497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22377,2018,2671,36827,10439,Budapest XVIII. kerület,100725,1650,47.4298307979262,19.2069099993847,Budapest,80580,0.611846,23363,0.388154,60190,1.576296
22378,2018,396,22524,15255,Budapest XX. kerület,64117,1650,47.436270236372,19.116183543411,Budapest,51294,0.613733,14176,0.386267,36700,1.588883
22379,2018,1096,27535,15736,Budapest XXI. kerület,74406,1650,47.4208517734888,19.0745350986312,Budapest,59525,0.629314,16219,0.370686,43754,1.697700
22380,2018,805,20372,15737,Budapest XXII. kerület,55282,1650,47.4115371838767,19.0064997652604,Budapest,44226,0.589741,14172,0.410259,34544,1.437482


In [ ]:
toSumlst = ['vote_NF', 'value', 'votePop_elig', 'vote_F', 'sumvote']
Budapest_only_sanity_check_vote = None

for colIdx in range(len(toSumlst)): 
  temp_df = Budapest_distr[['year', toSumlst[colIdx] ]].groupby(['year']).sum().reset_index()

  if colIdx == 0:
    Budapest_only_sanity_check_vote = temp_df

  else: 
    Budapest_only_sanity_check_vote = pd.merge(Budapest_only_sanity_check_vote, temp_df, on='year')



**Kitero** 

In [ ]:
geoname = pd.read_csv('gdrive/MyDrive/2020_21_2/Temalabor_1/geoname.csv', dtype=str)
geoname = trim_all_columns(geoname) #Using Function 2 
bpDistrictGeoname = geoname[geoname['name'].str.match('Budapest')].sort_values(by = ['name']) #the 0th one is just simply Budapest, and we just mine and sort all the name which contain Budapest districts
bpDistrictGeoname.reset_index(drop=True, inplace=True)

In [ ]:
bpDistrictGeoname[0:1]

,geonameid,name,geoid,wgs84_e_szelesseg,wgs84_k_hosszusag
0,16601,Budapest,1134,47.4825696830677,19.1592600789783


**Kitero Vege**

In [ ]:
Budapest_only_sanity_check_vote['settlement'] = 'Budapest'
Budapest_only_sanity_check_vote['settlementid'] = '1134'
Budapest_only_sanity_check_vote['geonameid'] = '16601'
Budapest_only_sanity_check_vote['county'] = 'Budapest'
Budapest_only_sanity_check_vote['datatypeid'] = '1650'
Budapest_only_sanity_check_vote['wgs84_e_szelesseg'] = '47.4825696830677'
Budapest_only_sanity_check_vote['wgs84_k_hosszusag'] = '19.1592600789783'
Budapest_only_sanity_check_vote['vote_rate_F'] = Budapest_only_sanity_check_vote['vote_F'] / (Budapest_only_sanity_check_vote['vote_F'] + Budapest_only_sanity_check_vote['vote_NF'])
Budapest_only_sanity_check_vote['vote_rate_NF'] = Budapest_only_sanity_check_vote['vote_NF'] / (Budapest_only_sanity_check_vote['vote_F'] + Budapest_only_sanity_check_vote['vote_NF'])
Budapest_only_sanity_check_vote['vote_rate_NF_per_F'] = Budapest_only_sanity_check_vote['vote_NF'] / Budapest_only_sanity_check_vote['vote_F']

In [ ]:
Budapest_only_sanity_check_vote

,year,vote_NF,value,votePop_elig,vote_F,sumvote,settlement,settlementid,geonameid,county,datatypeid,wgs84_e_szelesseg,wgs84_k_hosszusag,vote_rate_F,vote_rate_NF,vote_rate_NF_per_F
0,1990,925558,1956819,1565455,120439,1045997,Budapest,1134,16601,Budapest,1650,47.4825696830677,19.1592600789783,0.115143,0.884857,7.684870
1,1994,1045356,1890448,1512360,68509,1113865,Budapest,1134,16601,Budapest,1650,47.4825696830677,19.1592600789783,0.061506,0.938494,15.258667
2,1998,686358,1801483,1441187,247979,934337,Budapest,1134,16601,Budapest,1650,47.4825696830677,19.1592600789783,0.265406,0.734594,2.767807
3,2002,748013,1726872,1381495,345181,1093194,Budapest,1134,16601,Budapest,1650,47.4825696830677,19.1592600789783,0.315755,0.684245,2.167017
4,2006,659687,1684523,1347620,356982,1016669,Budapest,1134,16601,Budapest,1650,47.4825696830677,19.1592600789783,0.351129,0.648871,1.847956
5,2010,505789,1694900,1355921,436442,942231,Budapest,1134,16601,Budapest,1650,47.4825696830677,19.1592600789783,0.463201,0.536799,1.158892
6,2014,565706,1704649,1363718,354408,920114,Budapest,1134,16601,Budapest,1650,47.4825696830677,19.1592600789783,0.385178,0.614822,1.596200
7,2018,612617,1686361,1349088,377864,990481,Budapest,1134,16601,Budapest,1650,47.4825696830677,19.1592600789783,0.381495,0.618505,1.621263


In [ ]:
sanity_check_vote = sanity_check_vote[['year', 'county', 'settlement', 'settlementid', 'geonameid', 'datatypeid', 'value', 'votePop_elig', 'sumvote', 'vote_NF', 'vote_F', 'vote_rate_NF', 'vote_rate_F', 'vote_rate_NF_per_F', 'wgs84_e_szelesseg', 'wgs84_k_hosszusag']]
Budapest_only_sanity_check_vote = Budapest_only_sanity_check_vote[sanity_check_vote.columns]

df_electionVotes = pd.concat([sanity_check_vote, Budapest_only_sanity_check_vote])
df_electionVotes = df_electionVotes.replace('nan', np.nan)

df_electionVotes['vote_rate_NF'] = round(df_electionVotes['vote_rate_NF'], 4)
df_electionVotes['vote_rate_F'] = round(df_electionVotes['vote_rate_F'], 4)
df_electionVotes['vote_rate_NF_per_F'] = round(df_electionVotes['vote_rate_NF_per_F'], 4)

df_electionVotes.reset_index(drop=True, inplace=True)

In [ ]:
df_electionVotes

,year,county,settlement,settlementid,geonameid,datatypeid,value,votePop_elig,sumvote,vote_NF,vote_F,vote_rate_NF,vote_rate_F,vote_rate_NF_per_F,wgs84_e_szelesseg,wgs84_k_hosszusag
0,1990,Fejér megye,Aba,1512,9692,1650,4103,3282,1530,1456,74,0.9516,0.0484,19.6757,47.0343556977497,18.524754962635
1,1990,Baranya megye,Abaliget,1035,16239,1650,645,516,266,255,11,0.9586,0.0414,23.1818,46.1438461930372,18.1181108415909
2,1990,Heves megye,Abasár,2209,10360,1650,3347,2678,1719,1568,151,0.9122,0.0878,10.3841,47.8006871447644,20.0083983387857
3,1990,Borsod-Abaúj-Zemplén megye,Abaújalpár,1341,17191,1650,136,109,94,89,5,0.9468,0.0532,17.8000,48.3061466428872,21.234096519055
4,1990,Borsod-Abaúj-Zemplén megye,Abaújkér,2420,10500,1650,709,567,380,353,27,0.9289,0.0711,13.0741,48.3050378371762,21.2002207816169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25228,2002,Budapest,Budapest,1134,16601,1650,1726872,1381495,1093194,748013,345181,0.6842,0.3158,2.1670,47.4825696830677,19.1592600789783
25229,2006,Budapest,Budapest,1134,16601,1650,1684523,1347620,1016669,659687,356982,0.6489,0.3511,1.8480,47.4825696830677,19.1592600789783
25230,2010,Budapest,Budapest,1134,16601,1650,1694900,1355921,942231,505789,436442,0.5368,0.4632,1.1589,47.4825696830677,19.1592600789783
25231,2014,Budapest,Budapest,1134,16601,1650,1704649,1363718,920114,565706,354408,0.6148,0.3852,1.5962,47.4825696830677,19.1592600789783


In [ ]:
df_electionVotes.to_csv(path + 'electionVotes.csv', index = False)